<a href="https://colab.research.google.com/github/StefaniaCri/Depression_reddit/blob/main/creating_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install praw
!pip install unidecode
import pandas as pd
import praw
import pprint
from google.colab import drive
drive.mount("/content/drive")
from prawcore.exceptions import Forbidden
from unidecode import unidecode
from time import sleep

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.1 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


For scrapping the data from reddit we used praw. The details below are for connecting to the api

In [4]:
user_agent = "Scraper 1.0"
reddit = praw.Reddit (
client_id="FZdLATD3lbOQ6yHV9rQPZg",
client_secret="7KIgNJiqnx1vSc8h0nM5-8M3LMuygg",
user_agent=user_agent
)

Code snipet for using the api and extracting data from a specific subreddit


In [7]:
usernames = []
author_ids = []
titles = []
texts = []
permalink = []
is_depressed_list = []

counter_depressed = 0

for submission in reddit.subreddit('Men_RO').search('depresie',limit=None):
    try:
        is_depressed = True
        user = reddit.redditor(str(submission.author))
        if submission.author != 'AutoModerator':
            usernames.append(user.name)
            titles.append(unidecode(submission.title))
            texts.append(unidecode(submission.selftext))
            permalink.append(unidecode(submission.permalink))
            is_depressed_list.append(is_depressed)
        sleep(1)
    except Forbidden:
        print("Forbidden error occurred. Skipping this submission.")
        continue
# Create DataFrame
df = pd.DataFrame({
    'Username': usernames,
    'Title': titles,
    'Text': texts,
    'Link': permalink,
    'Is_Depressed': is_depressed_list
})

csv_filename = "reddit_data.csv"
df.to_csv(csv_filename, index=False)

print("CSV file saved successfully.")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



CSV file saved successfully.


Get the data manualy annotated and save it in a dataframe

In [8]:
import pandas as pd

csv_file_path = '/content/dataset_depresie - Sheet1 (1).csv'

df = pd.read_csv(csv_file_path)

print(df.head())

              Username                                  Title  \
0                Total                                    NaN   
1         Soft-Willing  Cum ati iesit din anxietate/depresie?   
2  Alices-Took-Xannies                        Depresie severa   
3            die4rythm      De ce tinerii sufera de depresie?   
4              Howll3h           Abuz de cuvantul "depresie"?   

                                                Text  \
0                                                NaN   
1  Pt cei care au fost in perioade nasoale rau si...   
2  Sunt urat, gras si am acnee \n Nu mi pot gasi ...   
3  Vad tot mai multe situatii in care tinerii 16-...   
4  De ceva vreme in majoritatea postarilor de pe ...   

                                                Link Is_Depressed  \
0                                                NaN          182   
1  /r/CasualRO/comments/14ppof8/cum_ati_iesit_din...         TRUE   
2      /r/CasualRO/comments/1028ixh/depresie_severa/         TRUE

In [9]:
import pandas as pd


users = set()

for index, row in df.iterrows():
    username = row['Username']
    is_depressed = row['Is_Depressed']
    is_diagnosed = row['Is_Diagnosed']

    users.add((username, is_depressed, is_diagnosed))

print(users)

{('This-Initiative5562', 'FALSE', 'FALSE'), ('wooolfieeee', 'FALSE', 'FALSE'), ('Pilluca', 'FALSE', 'FALSE'), ('ill_j', 'FALSE', 'FALSE'), ('Effective_Anxiety984', 'TRUE', 'FALSE'), ('Soft-Willing', 'TRUE', 'FALSE'), ('Jjjjjjjjjjjjoe', 'FALSE', 'FALSE'), ('heideroesslein', 'FALSE', 'FALSE'), ('FeistyRabbit9451', 'FALSE', 'FALSE'), ('pulicafranaru', 'TRUE', 'TRUE'), ('Firoscos', 'FALSE', 'FALSE'), ('LonelyetDeranged', 'TRUE', 'FALSE'), ('dangdaniel345', 'FALSE', 'FALSE'), ('stefiiherea', 'FALSE', 'FALSE'), ('TheAbbadon', 'FALSE', 'FALSE'), ('SeaCardiologist8273', 'TRUE', 'FALSE'), ('ck92ro', 'FALSE', 'FALSE'), ('Polite_user', 'FALSE', 'FALSE'), ('Killy48', 'TRUE', 'TRUE'), ('Personal_Math5610', 'TRUE', 'FALSE'), ('OrganicCap3978', 'FALSE', 'FALSE'), ('Pavilion17', 'FALSE', 'FALSE'), ('ConfusionMission881', 'TRUE', 'TRUE'), ('Keta47', 'FALSE', 'FALSE'), ('Marix019', 'FALSE', 'FALSE'), ('futacu_69', 'TRUE', 'FALSE'), ('OaseNegre13', 'FALSE', 'FALSE'), ('Great-Chipmunk-4557', 'FALSE', 'FAL

In [ ]:
!pip install langdetect
import json
import time
from langdetect import detect, LangDetectException

posts = []
train_posts = []
posts_with_true_depression = []
random_user = 1
for user, label1, label2 in users:
    print(user, label1, label2)
    time.sleep(1)

    try:
        user_obj = reddit.redditor(str(user))
        submissions = user_obj.submissions.new(limit=None)
        user_posts = []

        for submission in submissions:
            post_data = {
                "title": submission.title,
                "link": submission.url,
                "selftext": submission.selftext
            }

            try:
                # Detectează limba postării
                if detect(submission.selftext) == 'ro':
                    user_posts.append(post_data)
            except LangDetectException:
                # Dacă detectarea limbii eșuează, ignoră postarea
                continue

        user_dict = {
            "user": random_user,
            "label1": label1,
            "label2": label2,
            "posts": user_posts
        }

        if label1 == 'TRUE' and label2 == 'TRUE':
            posts_with_true_depression.append(user_dict)
        else:
            if label1 == 'TRUE' and label2 == 'FALSE':
                posts.append(user_dict)
            else:
                train_posts.append(user_dict)
        random_user += 1

    except Exception as e:
        print(f"Error processing user {user}: {e}")
        continue

with open("reddit_data_with_depression.json", "w") as json_file:
    json.dump(posts, json_file, indent=4)

with open("reddit_data_without_depression.json", "w") as json_file:
    json.dump(train_posts, json_file, indent=4)

with open("reddit_data_with_true_depression.json", "w") as json_file:
    json.dump(posts_with_true_depression, json_file, indent=4)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=c98128a50fc8297cb91c0eaeb17e45410850b96554465ad84058fb696b150e7d
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
This-Initiative5562 FALSE FALSE


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



wooolfieeee FALSE FALSE


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Pilluca FALSE FALSE


...

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Rehabilitationdoc TRUE TRUE


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



butnaraski FALSE FALSE


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

